In [ ]:
%pylab inline

In [ ]:
import sys
sys.path.append('../')

In [ ]:
from mxnet import nd 
from FracTALResUNet.models.changedetection.mantis.mantis_dn import *

In [ ]:
# D6nf32 example 
depth=6
norm_type='GroupNorm'
norm_groups=4
ftdepth=5
NClasses=2
nfilters_init=32
psp_depth=4
nheads_start=4


net = mantis_dn_cmtsk(nfilters_init=nfilters_init, NClasses=NClasses,depth=depth, ftdepth=ftdepth, psp_depth=psp_depth,norm_type=norm_type,norm_groups=norm_groups,nheads_start=nheads_start)
net.initialize()

In [ ]:
BatchSize = 4
img_size=256
NChannels = 3

input_img_1 = nd.random.uniform(shape=[BatchSize, NChannels, img_size, img_size])
input_img_2 = nd.random.uniform(shape=[BatchSize, NChannels, img_size, img_size])

## Example of forward operation:


In [ ]:
outs = net(input_img_1, input_img_2)

In [ ]:
# outs is a list of outputs, segmentation, boundary, distance. 
# Each has shape BatchSize, NClasses, img_size, img_size
for out in outs:
    print (out.shape)

### Example of performing backward with multitasking operation

In [ ]:
labels_segm = nd.random.uniform(shape=[BatchSize, NClasses, img_size, img_size])
labels_segm = labels_segm > 0.5

labels_bound = nd.random.uniform(shape=[BatchSize, NClasses, img_size, img_size])
labels_bound = labels_bound > 0.5

labels_dist = nd.random.uniform(shape=[BatchSize, NClasses, img_size, img_size])


labels = [labels_segm,labels_bound,labels_dist]
labels = nd.concat(*labels,dim=1)

In [ ]:
from mxnet import autograd
from ceecnet.nn.loss.mtsk_loss import *

In [ ]:
myMTSKL = mtsk_loss()

In [ ]:
with autograd.record():
    listOfPreds = net(input_img_1, input_img_2)
    loss = myMTSKL.loss(listOfPreds,labels)
    loss.backward()

In [ ]:
loss.shape

In [ ]:
loss